### Importing libraries...

In [1]:
import requests
from pymongo import MongoClient
import pandas as pd
import re
import json
from pandas.io.json import json_normalize
import os
import numpy as np

### Importing Companies Data

In [2]:
dbName = "companies.json"
mongodbURL = f"mongodb://localhost/{dbName}"

client = MongoClient(mongodbURL, connectTimeoutMS=2000, serverSelectionTimeoutMS=2000)
db = client.get_database()

### Verifying amount of companies of potential chosen country

In [3]:
data = db.companies.aggregate([{"$group":{"_id":"$offices.country_code","count":{"$sum":1}}}])
countries = list(data)

In [4]:
countries

[{'_id': ['USA', 'IND'], 'count': 35},
 {'_id': ['GBR', 'DEU'], 'count': 2},
 {'_id': ['USA', 'ISR'], 'count': 17},
 {'_id': ['USA', 'USA', 'USA', 'USA', 'USA', 'USA', 'USA'], 'count': 1},
 {'_id': ['FRA', 'FRA', 'MUS'], 'count': 1},
 {'_id': ['GBR', 'USA', 'AUS', 'ESP', 'NLD', 'HKG'], 'count': 1},
 {'_id': ['CHN', 'CHN'], 'count': 2},
 {'_id': ['TUN'], 'count': 2},
 {'_id': ['IND', 'IND', 'IND', 'IND'], 'count': 1},
 {'_id': ['USA', 'USA', 'USA', 'USA', 'IND'], 'count': 1},
 {'_id': ['JPN', 'GBR', 'JOR'], 'count': 1},
 {'_id': ['CAN', 'SGP', 'USA', 'USA', 'GBR', 'CHE', 'JPN'], 'count': 1},
 {'_id': ['GBR', 'IND'], 'count': 1},
 {'_id': ['DEU', 'TWN'], 'count': 2},
 {'_id': ['BEL', 'GBR'], 'count': 1},
 {'_id': ['GBR', 'GBR', 'DEU'], 'count': 1},
 {'_id': ['MAR', 'NLD'], 'count': 2},
 {'_id': ['GRC'], 'count': 14},
 {'_id': ['CAN', 'IND'], 'count': 1},
 {'_id': ['BIH'], 'count': 1},
 {'_id': ['USA', 'TWN', 'JPN', 'KOR', 'ARE'], 'count': 1},
 {'_id': ['USA', 'IND', 'IND'], 'count': 4},


In [5]:
county_counts=[]
for x in countries:
    county_counts.append(x['_id'])


In [6]:
county_counts = [c for n in county_counts for c in n]
county_counts

['USA',
 'IND',
 'GBR',
 'DEU',
 'USA',
 'ISR',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA',
 'FRA',
 'FRA',
 'MUS',
 'GBR',
 'USA',
 'AUS',
 'ESP',
 'NLD',
 'HKG',
 'CHN',
 'CHN',
 'TUN',
 'IND',
 'IND',
 'IND',
 'IND',
 'USA',
 'USA',
 'USA',
 'USA',
 'IND',
 'JPN',
 'GBR',
 'JOR',
 'CAN',
 'SGP',
 'USA',
 'USA',
 'GBR',
 'CHE',
 'JPN',
 'GBR',
 'IND',
 'DEU',
 'TWN',
 'BEL',
 'GBR',
 'GBR',
 'GBR',
 'DEU',
 'MAR',
 'NLD',
 'GRC',
 'CAN',
 'IND',
 'BIH',
 'USA',
 'TWN',
 'JPN',
 'KOR',
 'ARE',
 'USA',
 'IND',
 'IND',
 'USA',
 'CZE',
 'USA',
 'ESP',
 'BEL',
 'BEL',
 'DNK',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA',
 'NLD',
 'GBR',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA',
 'GBR',
 'USA',
 'USA',
 'USA',
 'GBR',
 'USA',
 'DEU',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA',
 'USA',
 'GBR',
 'SGP',
 'USA',
 'GBR',
 'LKA',
 'PRT',
 'PRT',
 'USA',
 'SWE',
 'USA',
 'FRA',
 'KOR',
 'USA',
 'CZE',
 'USA',
 'USA',
 'FRA',
 'USA',
 'USA',
 'FRA',
 'SWE',
 'NOR',
 'NLD',
 'SWE',
 'USA',


In [7]:
from collections import Counter
county_counts = Counter(county_counts)
{k: v for k, v in sorted(county_counts.items(), key=lambda item: item[1], reverse=True)}

{'USA': 775,
 'GBR': 257,
 'DEU': 116,
 'IND': 106,
 'FRA': 82,
 'AUS': 80,
 'CHN': 79,
 'ESP': 62,
 'CAN': 62,
 'JPN': 52,
 'SGP': 46,
 'NLD': 40,
 'SWE': 35,
 'CHE': 31,
 'BRA': 30,
 'ARG': 29,
 'ISR': 28,
 'ITA': 28,
 'HKG': 25,
 'IRL': 25,
 'RUS': 23,
 'TWN': 20,
 'BEL': 19,
 'MEX': 18,
 'NOR': 17,
 'DNK': 16,
 'KOR': 15,
 'NZL': 15,
 'ROM': 15,
 'ZAF': 15,
 'PHL': 15,
 'ARE': 14,
 'MYS': 14,
 'AUT': 14,
 'POL': 13,
 'PRT': 12,
 'FIN': 12,
 'UKR': 12,
 'CZE': 11,
 'TUR': 11,
 'CHL': 9,
 'VNM': 9,
 'BGR': 8,
 'GRC': 7,
 'THA': 6,
 'SVN': 6,
 'COL': 6,
 'EST': 5,
 'ANT': 5,
 'HUN': 5,
 'EGY': 5,
 'LKA': 4,
 'LVA': 4,
 'HRV': 4,
 'PAK': 4,
 'CYP': 4,
 'JOR': 3,
 'MAR': 3,
 'SAU': 3,
 'LTU': 3,
 'URY': 3,
 'NGA': 3,
 'BGD': 3,
 'IDN': 3,
 'MKD': 3,
 'LUX': 3,
 'MLT': 3,
 'MUS': 2,
 'BIH': 2,
 'LBN': 2,
 'ISL': 2,
 'GIB': 2,
 'PAN': 2,
 'VEN': 2,
 'PRK': 2,
 'CSS': 2,
 'NIC': 2,
 'ARA': 2,
 'VGB': 2,
 'KEN': 2,
 'QAT': 2,
 'UZB': 2,
 'TUN': 1,
 'DOM': 1,
 'SDN': 1,
 'PER': 1,
 'GHA': 1,

**Countries which companies I have more information of:**  
  
USA 🇺🇸  
GBR 🇬🇧  
DEU 🇩🇪   
IND 🇮🇪  
FRA 🇫🇷  

**Top Videogame development Countries:**

USA 🇺🇸 = 194  
GBR 🇬🇧 =  48  
DEU 🇩🇪 =   /  
IND 🇮🇪  =   /  
FRA 🇫🇷 =  23  

**Chosen Country:  
US** 🇺🇸 🇺🇸 🇺🇸 🇺🇸

# 1. Successful tech startups that have raised at least 1 Million dollars

**Query:**  
 - Country: USA  
 - Is a startup: founded after 2007)  
 - Tech categories: software,design,games_video,web, mobile
 - Raised at least 1 Million dollars 
 

In [8]:
query = {"$and":[
    {"offices.country_code":"USA"},
    {"founded_year":{"$gte":2007}},
    {"funding_rounds.raised_amount":{"$gte":1000000}},
    {"category_code":{"$in":["software","design","games_video","web", "mobile"]}},
]}
cur = db.companies.find(query)
df = pd.DataFrame(list(cur))
df

,_id,name,permalink,crunchbase_url,homepage_url,blog_url,blog_feed_url,twitter_username,category_code,number_of_employees,...,investments,acquisition,acquisitions,offices,milestones,ipo,video_embeds,screenshots,external_links,partners
0,52cdef7c4bab8bd675297da9,Mahalo,mahalo,http://www.crunchbase.com/company/mahalo,http://mahalo.com,http://blog.mahalo.com/,http://blog.mahalo.com/feed/,MahaloDotCom,web,40.0,...,[],None,[],"[{'description': '', 'address1': '3525 Eastham...",[],None,"[{'embed_code': '<embed src=""http://blip.tv/pl...",[],[],[]
1,52cdef7c4bab8bd675297dc2,AdaptiveBlue,adaptiveblue,http://www.crunchbase.com/company/adaptiveblue,http://www.getglue.com,http://blog.adaptiveblue.com,http://blog.adaptiveblue.com/?feed=rss2,GetGlue,games_video,15.0,...,[],None,[],"[{'description': '', 'address1': '131 Varick S...",[],None,"[{'embed_code': '<object width=""425"" height=""3...",[],[{'external_url': 'http://web20show.com/2009/1...,[]
2,52cdef7c4bab8bd675297dc0,Livestream,livestream,http://www.crunchbase.com/company/livestream,http://www.livestream.com,http://www.livestream.com/blog/,http://www.livestream.com/blog/?feed=rss2,livestream,games_video,120.0,...,[],"{'price_amount': None, 'price_currency_code': ...",[],"[{'description': 'Livestream HQ', 'address1': ...",[],None,"[{'embed_code': '<embed src=""http://blip.tv/pl...",[],[{'external_url': 'http://scobleizer.com/2007/...,[]
3,52cdef7c4bab8bd675297dc1,Ustream,ustream,http://www.crunchbase.com/company/ustream,http://www.ustream.tv,http://www.ustream.com/blog,http://www.ustream.com/blog/feed,USTREAM,games_video,250.0,...,[],None,[],"[{'description': 'San Francisco Office', 'addr...","[{'id': 6587, 'description': 'Ustream recogniz...",None,"[{'embed_code': '<object id=""flashObj"" width=""...","[{'available_sizes': [[[150, 103], 'assets/ima...",[{'external_url': 'http://facebook.com/ustream...,[]
4,52cdef7c4bab8bd675297dd1,SodaHead,sodahead,http://www.crunchbase.com/company/sodahead,http://sodahead.com,,,SodaHead,web,25.0,...,[],None,[],"[{'description': '', 'address1': '16161 Ventur...",[],None,[],"[{'available_sizes': [[[150, 131], 'assets/ima...",[{'external_url': 'http://www.prweb.com/releas...,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,52cdef7f4bab8bd67529c4f0,Culture Jam,culture-jam,http://www.crunchbase.com/company/culture-jam,http://site.promojam.com,http://promojam.tumblr.com/,,culturejam,web,15.0,...,[],None,[],"[{'description': '', 'address1': '2434 Lincoln...",[],None,[],[],[{'external_url': 'http://www.techcrunch.com/2...,[]
295,52cdef7f4bab8bd67529c55b,Clavis Technology,clavis-technology,http://www.crunchbase.com/company/clavis-techn...,http://www.clavistechnology.com,http://www.clavistechnology.com/news,http://www.clavistechnology.com/feed/,Clavistech,software,50.0,...,[],None,[],"[{'description': '', 'address1': '7th Floor,',...","[{'id': 7702, 'description': 'Clavis Secures S...",None,"[{'embed_code': '<p> <object classid=""clsid:d...","[{'available_sizes': [[[150, 92], 'assets/imag...",[{'external_url': 'http://the451group.com/repo...,[]
296,52cdef7f4bab8bd67529c59b,Luma International,luma-international,http://www.crunchbase.com/company/luma-interna...,http://www.lumacentral.com,http://blog.lumacentral.com,http://blog.lumacentral.com/atom.xml,lumacentral,software,40.0,...,[],None,[],"[{'description': 'Headquarters', 'address1': '...",[],None,[],"[{'available_sizes': [[[150, 114], 'assets/ima...",[],[]
297,52cdef7f4bab8bd67529c5b8,Continuity Engine,continuity-engine,http://www.crunchbase.com/company/continuity-e...,http://www.continuity.net,http://continuityengine.blogspot.com/,,ContinuityEng,software,11.0,...,[],None,"[{'price_amount': None, 'price_currency_code':...","[{'description': 'HQ', 'address1': '150 Munson...",[],None,[],"[{'available_sizes': [[[150, 72], 'assets/imag...",[],[]


In [9]:
df.columns

Index(['_id', 'name', 'permalink', 'crunchbase_url', 'homepage_url',
       'blog_url', 'blog_feed_url', 'twitter_username', 'category_code',
       'number_of_employees', 'founded_year', 'founded_month', 'founded_day',
       'deadpooled_year', 'deadpooled_month', 'deadpooled_day',
       'deadpooled_url', 'tag_list', 'alias_list', 'email_address',
       'phone_number', 'description', 'created_at', 'updated_at', 'overview',
       'image', 'products', 'relationships', 'competitions', 'providerships',
       'total_money_raised', 'funding_rounds', 'investments', 'acquisition',
       'acquisitions', 'offices', 'milestones', 'ipo', 'video_embeds',
       'screenshots', 'external_links', 'partners'],
      dtype='object')

In [10]:
offices = df[["name", "offices"]].explode("offices")
offices

,name,offices
0,Mahalo,"{'description': '', 'address1': '3525 Eastham ..."
1,AdaptiveBlue,"{'description': '', 'address1': '131 Varick St..."
2,Livestream,"{'description': 'Livestream HQ', 'address1': '..."
3,Ustream,"{'description': 'San Francisco Office', 'addre..."
3,Ustream,"{'description': 'Los Angeles Office', 'address..."
...,...,...
296,Luma International,"{'description': 'HQ North America', 'address1'..."
296,Luma International,"{'description': 'Office France', 'address1': '..."
296,Luma International,"{'description': 'Office The Netherlands', 'add..."
297,Continuity Engine,"{'description': 'HQ', 'address1': '150 Munson ..."


In [11]:
offices = json_normalize(json.loads(offices.to_json(orient="records")))
offices = offices[["name", "offices.latitude", "offices.longitude", "offices.state_code"]]
offices

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """Entry point for launching an IPython kernel.


,name,offices.latitude,offices.longitude,offices.state_code
0,Mahalo,34.017606,-118.487267,CA
1,AdaptiveBlue,40.801358,-74.337200,NY
2,Livestream,40.726155,-73.995625,NY
3,Ustream,37.392936,-122.079480,CA
4,Ustream,NaN,NaN,CA
...,...,...,...,...
388,Luma International,NaN,NaN,CA
389,Luma International,NaN,NaN,None
390,Luma International,NaN,NaN,None
391,Continuity Engine,41.327918,-73.243552,CT


In [12]:
offices["offices.state_code"].value_counts()

CA    184
NY     50
MA     23
WA     11
CO      9
TX      9
VA      7
NC      5
IL      5
FL      5
PA      4
CT      3
OR      3
GA      3
UT      3
MD      2
MN      2
AZ      2
NJ      2
SC      1
NH      1
DC      1
RI      1
LA      1
ME      1
KY      1
Name: offices.state_code, dtype: int64

### Chosen state: California

In [13]:
offices = offices.loc[offices["offices.state_code"] == "CA"]
offices = offices.dropna()
offices = offices.reset_index()
offices = offices.drop(columns = ["index"])

In [14]:
offices

,name,offices.latitude,offices.longitude,offices.state_code
0,Mahalo,34.017606,-118.487267,CA
1,Ustream,37.392936,-122.079480,CA
2,SodaHead,37.269175,-119.306607,CA
3,Funny Or Die,37.421621,-122.137444,CA
4,Seesmic,37.775196,-122.419204,CA
...,...,...,...,...
138,Global Fitness Media,34.054318,-118.462876,CA
139,CrowdGather,34.168955,-118.575293,CA
140,High Gear Media,37.445335,-122.158330,CA
141,Teambox,41.406335,2.192023,CA


In [15]:
offices = offices.rename(columns={"name": "Name", "offices.latitude": "Latitude", "offices.longitude": "Longitude", "offices.state_code": "State Code"})

In [16]:
offices

,Name,Latitude,Longitude,State Code
0,Mahalo,34.017606,-118.487267,CA
1,Ustream,37.392936,-122.079480,CA
2,SodaHead,37.269175,-119.306607,CA
3,Funny Or Die,37.421621,-122.137444,CA
4,Seesmic,37.775196,-122.419204,CA
...,...,...,...,...
138,Global Fitness Media,34.054318,-118.462876,CA
139,CrowdGather,34.168955,-118.575293,CA
140,High Gear Media,37.445335,-122.158330,CA
141,Teambox,41.406335,2.192023,CA


## Google API

In [17]:
from dotenv import load_dotenv
load_dotenv()
apiKey = os.getenv("google_key")
url = f"https://maps.googleapis.com/maps/api/place/textsearch/json?query=starbucks+in+CA&key={apiKey}"
res = requests.get(url)


In [18]:
res = res.json()

In [19]:
from dotenv import load_dotenv
load_dotenv()
apiKey = os.getenv("google_key")
url = "https://maps.googleapis.com/maps/api/place/textsearch/json"
#query=starbucks+in+CA&key={apiKey}&pageToken=res['next_page_token']"
res = requests.get(url)
res = res.json()

In [20]:
def getData(query, page_token=None):
    url = "https://maps.googleapis.com/maps/api/place/textsearch/json"
    params = {"key":os.getenv("google_key"), "query":query}
    if page_token:
        params["pageToken"]=page_token
    res = requests.get(url, params=params)
    res = res.json()
    results = res["results"]
    if 'next_page_token' in res.keys():
        page_token = res['next_page_token']
        results=[results, page_token]
    else: 
        results=[results,False]
    return results

In [21]:
def getLocations(query, limit=50):
    data = []
    page_token = None
    while len(data)<limit:
        results, page_token = getData(query, page_token)
        data+=results
        if not page_token: 
            break
    return data

##  Función para calcular distancias

In [22]:
def FindDistances(req):
    d=[]
    for i in offices.index:
        d_list=[]
        for ind in req.index: 
            x = req['Latitude'][ind]
            y = req['Longitude'][ind] 
            distance = (((x-(offices['Latitude'][i]))**2)+((y-(offices['Longitude'][i]))**2))**(1/2)
            d_list.append(distance)
        d.append(min(d_list))
    offices['New'] = d

## Starbucks

In [25]:
data = getLocations("starbucks in CA", 100)
dataset = pd.DataFrame(data)

In [27]:
starbucks = []
for st in dataset["geometry"]:
    new_dict={}
    new_dict["Latitude"]= st['location']['lat']
    new_dict["Longitude"]= st['location']['lng']
    starbucks.append(new_dict)

In [28]:
starbucks = pd.DataFrame(starbucks)
starbucks

,Latitude,Longitude
0,34.072428,-117.881891
1,37.291159,-121.989834
2,37.762265,-121.433395
3,38.714409,-120.839673
4,38.762384,-120.576595
...,...,...
95,37.426867,-122.144091
96,35.368910,-119.018524
97,38.013442,-121.868315
98,33.649162,-117.830863


In [29]:
def transformToGeoPoint(s):
    if np.isnan(s.Latitude) or np.isnan(s.Longitude):
        return None
    return {
        "type":"Point",
        "coordinates":[s.Longitude, s.Latitude]
    }
starbucks["geopoint"] = starbucks.apply(transformToGeoPoint, axis=1)
starbucks

,Latitude,Longitude,geopoint
0,34.072428,-117.881891,"{'type': 'Point', 'coordinates': [-117.881891,..."
1,37.291159,-121.989834,"{'type': 'Point', 'coordinates': [-121.9898337..."
2,37.762265,-121.433395,"{'type': 'Point', 'coordinates': [-121.4333954..."
3,38.714409,-120.839673,"{'type': 'Point', 'coordinates': [-120.839673,..."
4,38.762384,-120.576595,"{'type': 'Point', 'coordinates': [-120.5765954..."
...,...,...,...
95,37.426867,-122.144091,"{'type': 'Point', 'coordinates': [-122.144091,..."
96,35.368910,-119.018524,"{'type': 'Point', 'coordinates': [-119.0185241..."
97,38.013442,-121.868315,"{'type': 'Point', 'coordinates': [-121.868315,..."
98,33.649162,-117.830863,"{'type': 'Point', 'coordinates': [-117.8308631..."


In [30]:
FindDistances(starbucks)

In [31]:
offices = offices.rename(columns={"New":"Starbucks"})

## Bars

In [ ]:
data = getLocations("bars in CA", 100)
dataset = pd.DataFrame(data)